1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
import pandas as pd

In [2]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [3]:
file_name = "./data/data_000637.txt"
N = 0
while(N < 10000):
    try:
        N = int(input("Insert a number larger than 10000:\n"))
    except Exception:
        print("Input not valid") 
data = pd.read_csv(file_name,nrows=N)
data 

Insert a number larger than 10000:
131000


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
130995,1,1,4,3869201469,1734,29
130996,1,1,139,3869201469,1743,0
130997,1,1,5,3869201469,1741,1
130998,1,1,2,3869201469,1741,3


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [4]:
maxBX = data['BX_COUNTER'].max()
maxBX

3563

3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

*Hint:* introduce an offset to the absolute time such that the start of the data acquisition (i.e. the first entry) is zero.

In [5]:
data3 = data.copy()
data3['TIME_MEAS'] = pd.to_datetime(data3['BX_COUNTER'] + maxBX * data3['ORBIT_CNT'] + data3['TDC_MEAS'])

data3

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME_MEAS
0,1,0,123,3869200167,2374,26,1970-01-01 03:49:45.960197421
1,1,0,124,3869200167,2374,27,1970-01-01 03:49:45.960197422
2,1,0,63,3869200167,2553,28,1970-01-01 03:49:45.960197602
3,1,0,64,3869200167,2558,19,1970-01-01 03:49:45.960197598
4,1,0,64,3869200167,2760,25,1970-01-01 03:49:45.960197806
...,...,...,...,...,...,...,...
130995,1,1,4,3869201469,1734,29,1970-01-01 03:49:45.964835810
130996,1,1,139,3869201469,1743,0,1970-01-01 03:49:45.964835790
130997,1,1,5,3869201469,1741,1,1970-01-01 03:49:45.964835789
130998,1,1,2,3869201469,1741,3,1970-01-01 03:49:45.964835791


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [6]:

data['TIME_MEAS'] = pd.to_datetime(data3['BX_COUNTER'] + maxBX * data3['ORBIT_CNT'] + data3['TDC_MEAS'])
print(pd.to_timedelta(data['TIME_MEAS'].max()-data['TIME_MEAS'].min()))

0 days 00:00:00.004638455


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [7]:
data5 = data.copy()
data5 = data5.groupby('TDC_CHANNEL')['TDC_CHANNEL'].count().nlargest(3)
data5

TDC_CHANNEL
139    10911
64      6543
63      6407
Name: TDC_CHANNEL, dtype: int64

6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [8]:
data.ORBIT_CNT.nunique()

1303

7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [9]:
data[data['TDC_CHANNEL']==139].ORBIT_CNT.nunique()

1302

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [15]:
df_fpga0 = data[data['FPGA'].isin([0])]
series0 = df_fpga0.groupby('TDC_CHANNEL')['TDC_CHANNEL'].count()

df_fpga1 = data[data['FPGA'].isin([1])]
series1 = df_fpga1.groupby('TDC_CHANNEL')['TDC_CHANNEL'].count()
print(series0)
print(series1)

TDC_CHANNEL
1       120
2       143
3       152
4       210
5       158
       ... 
129       1
130       5
137       4
138       4
139    7648
Name: TDC_CHANNEL, Length: 122, dtype: int64
TDC_CHANNEL
1      2816
2      3251
3      2177
4      2620
5      1504
       ... 
129       4
130       5
137       4
138       4
139    3263
Name: TDC_CHANNEL, Length: 132, dtype: int64


9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.